<a href="https://colab.research.google.com/github/serepina/Human_Detector_Comparison/blob/main/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**YOLOv3 OpenCV를 사용하여서 Video 결과 얻기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install --user opencv-contrib-python

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import os
import time

cv2.face.LBPHFaceRecognizer_create() 

classes = ["person", "bicycle", "car", "motorcycle",
            "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant",
            "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
            "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
            "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis",
            "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard",
            "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife",
            "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
            "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table",
            "toilet", "tv", "laptop", "mouse", "remote", "keyboard",
            "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
            "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush" ]

weightsPath = os.path.sep.join([os.path.abspath("/content/drive/MyDrive/Colab Notebooks/YOLOv3"), "yolov4.weights"])
configPath = os.path.sep.join([os.path.abspath("/content/drive/MyDrive/Colab Notebooks/YOLOv3"), "yolov4-csp.cfg"])
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/test1.mp4')

fps = int(cap.get(cv2.CAP_PROP_FPS))
width  = int(cap.get(3)) # float
height = int(cap.get(4)) # float

fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
vout = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/video4_1.mp4', fourcc, fps, (width,height))

start = time.time()  # 시작 시간 저장
while (cap.isOpened()):
  ret, img = cap.read()

  if ret:
    height, width, channels = img.shape

    image = img
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.1:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 10), font, 1, color, 2)
            #print(label,confidences[i],x,y,w,h)

    vout.write(image)

  else:
    print("time :", time.time() - start)
    break

cap.release()
cv2.destroyAllWindows()

error: ignored